## Libraries

In [1]:
import pandas as pd
import numpy as np
import sys
import getopt

In [9]:
def main(argv):
    idexfile = 'IDEXTrades.csv'
    decimalsfile = 'token_decimals.json'
    outputfile = 'IDEX-preprocessed.csv'
    short_options = "hi:d:o:"
    long_options = ["help", "idexfile=", "decimalsfile=" "outputfile="]
    
    try:
        opts, args = getopt.getopt(argv, short_options, long_options)
    except getopt.GetoptError:
        print('IDEXtrades_preprocessing.ipynb -i <idexfile> -d <decimalsfile> -o <outputfile>')
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print('IDEXtrades_preprocessing.ipynb -i <idexfile> -d <decimalsfile> -o <outputfile>')
            sys.exit()
        elif opt in ("-i", "--idexfile"):
            idexfile = arg
        elif opt in ("-d", "--decimalsfile"):
            decimalsfile = arg
        elif opt in ("-o", "--outputfile"):
            outputfile = arg
  # if any argument is missing, stop
    if (idexfile == '' or decimalsfile == '' or outputfile == ''):
        print('Warning: missing arguments. Arguments need to be supplied as follows:')
        print('IDEXtrades_preprocessing.ipynb -i <idexfile> -d <decimalsfile> -o <outputfile>')
        sys.exit(2)
        
    trades = pd.read_csv(idexfile, header=0)
    orig_len = len(trades)
    
    token_decimals = pd.read_json(decimalsfile, orient="index")
    token_decimals.reset_index(inplace=True)
    token_decimals.drop(labels=["index", "name", "slug"], axis=1, inplace=True)
    
    IDEX_tokens = pd.DataFrame(pd.unique(pd.concat([trades['tokenBuy'], trades['tokenSell']], axis=0)),
                             columns = ['address'])
    missing_IDEX_tokens = set(IDEX_tokens['address'].values) - set(token_decimals['address'].values)
 
    if(len(missing_IDEX_tokens) > 0):
        print("Warning:", len(missing_IDEX_tokens), "tokens in IDEX data have no decimal information in respective file. " +
      "As a default, 18 will be taken as their decimal.")
    token_decimals_all = pd.merge(token_decimals, IDEX_tokens,
                                how = 'right') # keep only IDEX tokens
    token_decimals_all[['decimals']] = token_decimals_all[['decimals']].fillna(value = 18)
 
    trades_real = trades.merge(token_decimals_all, how="left", left_on="tokenBuy", right_on="address")

    trades_real.drop(labels=["address"], axis=1, inplace=True)

    trades_real['amountBuyReal'] = trades_real['amountBuy']
    trades_real['amountBoughtReal'] = trades_real['amount']
    trades_real = trades_real.astype({'amountBuyReal': float, 'amountBoughtReal': float})
    trades_real['amountBuyReal'] = trades_real['amountBuyReal'].divide(10**trades_real['decimals'])
    trades_real['amountBoughtReal'] = trades_real['amountBoughtReal'].divide(10**trades_real['decimals'])
    trades_real.drop(labels=['decimals'],
                  axis=1, inplace=True)

    trades_real = trades_real.merge(token_decimals_all, how="left", left_on="tokenSell", right_on="address")
    trades_real.drop(labels=["address"], axis=1, inplace=True)
    trades_real['amountSellReal'] = trades_real['amountSell']
    trades_real = trades_real.astype({'amountSellReal': float}) 
    trades_real['amountSellReal'] = trades_real['amountSellReal'].divide(10**trades_real['decimals'])
    trades_real.drop(labels=['decimals'],
                  axis=1, inplace=True)
    trades_real['price'] = trades_real['amountSellReal'].divide(trades_real['amountBuyReal'])
    trades_real['amountSoldReal'] = trades_real['amountBoughtReal'].mul(trades_real['price'])

    trades_real = trades_real.astype({'feeMake': float,
                                    'feeTake': float})
    trades_real['feeMake'] = trades_real['feeMake'].divide(10**18)
    trades_real['feeTake'] = trades_real['feeTake'].divide(10**18)
    
    trades_real = trades_real.rename(columns={'transaction_hash':'transactionHash',
                                            'block_number':'blockNumber',
                                            'gas_price':'gasPrice'
                                           })

    trades_real = trades_real[['blockNumber',
                             'timestamp',
                             'transactionHash',
                             'status',
                             'maker',
                             'taker',
                             'tokenBuy',
                             'tokenSell',
                             'amountBuyReal',
                             'amountBoughtReal',
                             'amountSellReal',
                             'amountSoldReal',
                             'price',
                             'feeMake',
                             'feeTake',
                             'gas',
                             'gasPrice',
                             'nonce',
                             'tradeNonce',
                             'expires']]
    if len(trades_real) != orig_len:
        print("Warning: dropped", orig_len - len(trades_real), "rows during Preprocessing.")
    trades_real.to_csv(outputfile, index=False)
    print("Info: saved file to " + outputfile)
    
if __name__ == "__main__":
    main(sys.argv[1:])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



IDEXtrades_preprocessing.ipynb -i <idexfile> -d <decimalsfile> -o <outputfile>
Traceback (most recent call last):
  File "C:\Users\chess\AppData\Local\Temp/ipykernel_19684/1838720677.py", line 9, in main
    opts, args = getopt.getopt(argv, short_options, long_options)
  File "C:\Users\chess\anaconda3\lib\getopt.py", line 95, in getopt
    opts, args = do_shorts(opts, args[0][1:], shortopts, args[1:])
  File "C:\Users\chess\anaconda3\lib\getopt.py", line 195, in do_shorts
    if short_has_arg(opt, shortopts):
  File "C:\Users\chess\anaconda3\lib\getopt.py", line 211, in short_has_arg
    raise GetoptError(_('option -%s not recognized') % opt, opt)
getopt.GetoptError: option -f not recognized

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\chess\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\chess\AppData\

TypeError: object of type 'NoneType' has no len()

In [8]:
%tb

TypeError: object of type 'NoneType' has no len()

## Decimals

Info: read decimal information for 515 tokens from Decimals file.


## Transform and Merge
## Tokenbuy

Info: converted column 'amountBuy' to 'amountBuyReal' as the order amount of 'tokenBuy', and 'amount' to 'amountBoughtReal' as the trade amount of 'tokenBuy', according to token decimals.


## Tokensell

Info: converted column 'amountSell' to 'amountSellReal' as the order amount of 'tokenSell', according to token decimals.


## Price and amount sold

Info: computed new column 'price' as price of tokenBuy in units of tokenSell, and column 'amountSoldReal' as the trade amount of 'tokenSell'.


## FEES

Info: divided columns 'feeMake' and 'feeTake' by 10^18. For more information, see https://gist.github.com/raypulver/2f318db5dc497cab8019d3ae391af1d2.


## EXPORT

## SAVE AS CSV

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Info: saved file to IDEXtrades_preprocessing.csv
IDEXtrades_preprocessing.py -i <idexfile> -d <decimalsfile> -o <outputfile>
Traceback (most recent call last):
  File "C:\Users\chess\AppData\Local\Temp/ipykernel_27456/2172055478.py", line 9, in main
    opts, args = getopt.getopt(argv, short_options, long_options)
  File "C:\Users\chess\anaconda3\lib\getopt.py", line 95, in getopt
    opts, args = do_shorts(opts, args[0][1:], shortopts, args[1:])
  File "C:\Users\chess\anaconda3\lib\getopt.py", line 195, in do_shorts
    if short_has_arg(opt, shortopts):
  File "C:\Users\chess\anaconda3\lib\getopt.py", line 211, in short_has_arg
    raise GetoptError(_('option -%s not recognized') % opt, opt)
getopt.GetoptError: option -f not recognized

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\chess\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns,

TypeError: object of type 'NoneType' has no len()